In [ ]:
import os
import requests
import tensorflow as tf
import zipfile
from PIL import Image
import io

class Classifier:
    def __init__(self):
        self.model = None
        self.dataset = None
        self.users = {}

    def _download_file(self, url, destination_path):
        if "drive.google.com" in url:
            file_id = url.split('/d/')[1].split('/')[0]
            url = f"https://drive.google.com/uc?export=download&id={file_id}"
        response = requests.get(url, stream=True)
        with open(destination_path, 'wb') as f:
            for chunk in response.iter_content(chunk_size=128):
                f.write(chunk)

    def _extract_zip(self, file_path, extract_path):
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)

    def _filter_images(self, directory):
        for subdir, dirs, files in os.walk(directory):
            for file in files:
                file_path = os.path.join(subdir, file)
                try:
                    Image.open(file_path)
                except (IOError, SyntaxError):
                    #print(f'Removing non-image file: {file_path}')
                    os.remove(file_path)

    def getModel(self, model_url):
        model_path = 'downloaded_model'
        self._download_file(model_url, model_path)
        self.model = tf.keras.models.load_model(model_path)

    def getDataset(self, dataset_url):
        dataset_zip_path = 'downloaded_dataset.zip'
        dataset_extract_path = 'dataset'
        self._download_file(dataset_url, dataset_zip_path)
        self._extract_zip(dataset_zip_path, dataset_extract_path)
        self._filter_images(dataset_extract_path)
        self.dataset = tf.keras.preprocessing.image_dataset_from_directory(
            dataset_extract_path,
            color_mode='grayscale',  # Convert images to grayscale
            image_size=(28, 28),  # Resize images to 28x28
            batch_size=32  # Adjust based on your specific requirements
        )

    def setMinReward(self, user_address, min_reward):
        if user_address not in self.users:
            self.users[user_address] = {'min_reward': None, 'labels': []}
        self.users[user_address]['min_reward'] = min_reward

    def classify(self, user_address):
        if user_address not in self.users or self.model is None or self.dataset is None:
            raise ValueError("User not found or model/dataset not loaded")

        predictions = []
        for images, _ in self.dataset:
            preds = self.model.predict(images)
            predictions.extend(tf.argmax(preds, axis=1).numpy())

        self.users[user_address]['labels'] = predictions
        return predictions

# Example usage
classifier = Classifier()
classifier.getModel('https://drive.google.com/file/d/1elfSiCLAL2Wh9Qo33biMl_NSVVx_fMuS/view?usp=sharing')
classifier.getDataset('https://drive.google.com/file/d/1NUuR_01a64nXmTcvtouzTBkDch6_cMKZ/view?usp=sharing')
classifier.setMinReward('user123', 0.01)
labels = classifier.classify('user123')



Found 980 files belonging to 2 classes.
1/1 [==============================] - 0s 115ms/step


In [ ]:
classifier = Classifier()
classifier.getModel('https://drive.google.com/file/d/1elfSiCLAL2Wh9Qo33biMl_NSVVx_fMuS/view?usp=sharing')
classifier.getDataset('https://drive.google.com/file/d/1NUuR_01a64nXmTcvtouzTBkDch6_cMKZ/view?usp=sharing')
classifier.setMinReward('user123', 0.01)
labels = classifier.classify('user123')

Found 980 files belonging to 2 classes.
1/1 [==============================] - 0s 121ms/step


In [9]:
%pip install tensorflow==2.15

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 37.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 40.1 MB/s eta 0:00:00
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: google-auth-oauthlib
    Found existing installation: google-auth-oauthlib 0.4.6
    Uninstalling google-auth-oauthlib-0.4.6:
      Successfully uninstalled google-auth-oauthlib-0.4.6
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.0
    Uninstalling te

In [2]:
import requests
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM

class TextGenerator:
    def __init__(self, model_name='pszemraj/opt-350m-email-generation'):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForCausalLM.from_pretrained(model_name)
        self.generator = pipeline('text-generation', model=self.model, tokenizer=self.tokenizer)
        self.users = {}

    def setMinReward(self, user_address, min_reward):
        if user_address not in self.users:
            self.users[user_address] = {'min_reward': None, 'texts': []}
        self.users[user_address]['min_reward'] = min_reward

    def generateText(self, user_address, prompt="Here is a starting sentence:", max_length=50):
        if user_address not in self.users:
            raise ValueError("User not found")

        generated_texts = self.generator(prompt, max_length=max_length, num_return_sequences=1)
        generated_text = generated_texts[0]['generated_text']
        self.users[user_address]['texts'].append(generated_text)
        return generated_text



In [3]:
# Example usage
textGenerator = TextGenerator()
textGenerator.setMinReward('user123', 0.01)
generated_text = textGenerator.generateText('user123', "The future of AI is", 50)
print(generated_text)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/894 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.32G [00:00<?, ?B/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


The future of AI is here.
The future of computing is here.
The future of data is here.
And the future of business is here.
We are excited to announce the launch of the new version of the AI Marketplace.



In [3]:
generated_text = textGenerator.generateText('user123', "i need you to finish by", 50)
print(generated_text)

i need you to finish by <NUMBER>:<NUMBER> pm today.
i will be out of the office until <NUMBER>:<NUMBER> pm.
i will be checking voice mail.
i will be checking email


In [4]:
generated_text = textGenerator.generateText('user123', "i need you to finish by", 50)
print(generated_text)

i need you to finish by <NUMBER>:<NUMBER> pm today.
i will be out of the office until <NUMBER>:<NUMBER> pm.
i will be checking voice mail.
i will be checking email


In [5]:
generated_text = textGenerator.generateText('user123', "i need you to finish by", 100)
print(generated_text)

i need you to finish by <NUMBER>:<NUMBER> pm today.
i will be out of the office until <NUMBER>:<NUMBER> pm.
i will be checking voice mail.
i will be checking email.
i will be checking voice mail.
i will be checking email.
i will be checking voice mail.
i will be checking email.
i will be checking email.
i will be checking voice mail.
i will be


In [6]:
generated_text = textGenerator.generateText('user123', "when should we get to the", 100)
print(generated_text)

when should we get to the point where we can start using the new system?
I would like to start using it by the end of next week.
Thanks, Jim.
DF <NUMBER>-<NUMBER>-<NUMBER> <NUMBER>:<NUMBER> PM To: Jim Derrick/HOU/ECT@ECT cc: Subject: New System for EOL Trading <NUMBER>-<NUMBER>-<NUMBER> <NUMBER>:<
